In [1]:
import pandas as pd
import re
from pulp import *
from pathlib import Path

In [2]:
pwd = Path().cwd()
data_dir = pwd / "data"

In [3]:
df = pd.read_csv(data_dir / "players_raw.csv")
df_teams = pd.read_csv(data_dir / "teams.csv")

In [4]:
df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,...,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
0,1,3,292,None,None,3,58822,2,NaN,0,...,80,48,0,0,0,0,0.0,10.7,Cédric,3
1,0,0,69,None,None,1,80201,None,NaN,0,...,27,10,0,0,0,0,0.0,2.2,Leno,0
2,2,3,334,None,None,6,84450,None,NaN,0,...,85,60,0,0,0,0,0.0,12.0,Xhaka,10
3,2,0,156,None,None,0,153256,None,NaN,0,...,145,27,0,0,0,0,0.0,6.0,Elneny,1
4,0,2,152,None,None,2,156074,None,NaN,0,...,112,29,0,0,0,0,0.0,6.4,Holding,2


In [5]:
teams_dict = pd.Series(df_teams.short_name.values,index=df_teams.id).to_dict()
pos_dict = {1: 'GK', 2: 'DEF', 3: 'MID', 4: 'FWD'}

In [6]:
df_modify = df.copy()

In [7]:
df_modify["name"] = df["first_name"] + " " + df["second_name"]

In [8]:
df_modify.drop(columns=["first_name", "second_name"], axis=1, inplace=True)

In [9]:
df_modify.columns

Index(['assists', 'bonus', 'bps', 'chance_of_playing_next_round',
       'chance_of_playing_this_round', 'clean_sheets', 'code',
       'corners_and_indirect_freekicks_order',
       'corners_and_indirect_freekicks_text', 'cost_change_event',
       'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall',
       'creativity', 'creativity_rank', 'creativity_rank_type',
       'direct_freekicks_order', 'direct_freekicks_text', 'dreamteam_count',
       'element_type', 'ep_next', 'ep_this', 'event_points', 'form',
       'goals_conceded', 'goals_scored', 'ict_index', 'ict_index_rank',
       'ict_index_rank_type', 'id', 'in_dreamteam', 'influence',
       'influence_rank', 'influence_rank_type', 'minutes', 'news',
       'news_added', 'now_cost', 'own_goals', 'penalties_missed',
       'penalties_order', 'penalties_saved', 'penalties_text', 'photo',
       'points_per_game', 'red_cards', 'saves', 'selected_by_percent',
       'special', 'squad_number', 'status', 'team', 't

In [10]:
df_modify.rename({"element_type": "position", "now_cost": "value"}, inplace=True, axis=1)

In [11]:
features = df_modify[['name', "team", 'position', 'total_points', 'value']]

In [12]:
features

,name,team,position,total_points,value
0,Cédric Alves Soares,1,2,48,45
1,Bernd Leno,1,1,10,45
2,Granit Xhaka,1,3,60,50
3,Mohamed Elneny,1,3,27,45
4,Rob Holding,1,2,29,45
...,...,...,...,...,...
531,Matija Šarkić,20,1,0,40
532,Morgan Gibbs-White,20,3,1,55
533,Yerson Mosquera Valdelamar,20,2,0,40
534,João Filipe Iria Santos Moutinho,20,3,96,50


In [13]:
features["team"] = features["team"].map(teams_dict)
features["position"] = features["position"].map(pos_dict)

C:\Users\user\AppData\Local\Temp\ipykernel_15960\1191323582.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features["team"] = features["team"].map(teams_dict)
C:\Users\user\AppData\Local\Temp\ipykernel_15960\1191323582.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features["position"] = features["position"].map(pos_dict)


In [14]:
features

,name,team,position,total_points,value
0,Cédric Alves Soares,ARS,DEF,48,45
1,Bernd Leno,ARS,GK,10,45
2,Granit Xhaka,ARS,MID,60,50
3,Mohamed Elneny,ARS,MID,27,45
4,Rob Holding,ARS,DEF,29,45
...,...,...,...,...,...
531,Matija Šarkić,WOL,GK,0,40
532,Morgan Gibbs-White,WOL,MID,1,55
533,Yerson Mosquera Valdelamar,WOL,DEF,0,40
534,João Filipe Iria Santos Moutinho,WOL,MID,96,50


In [15]:
POS = features.position.unique()
CLUBS = features.team.unique()
BUDGET = 1000
pos_available = {
    'DEF': 5,
    'FWD': 3,
    'MID': 5,
    'GK': 2,
}

# Initialize Variables
names = [features.name[i] for i in features.index]
teams = [features.team[i] for i in features.index]
positions = [features.position[i] for i in features.index]
prices = [features.value[i] for i in features.index]
points = [features.total_points[i] for i in features.index]
players = [LpVariable("player_" + str(i), cat="Binary") for i in features.index]

In [16]:
# Initialize the problem
prob = LpProblem("FPL_Player_Choices", LpMaximize)

In [17]:
# Define the objective
prob += lpSum(players[i] * points[i] for i in range(len(features))) # Objective

In [18]:
# Build the constraints
prob += lpSum(players[i] * features.value[features.index[i]] for i in range(len(features))) <= BUDGET # Budget Limit

for pos in POS:
  prob += lpSum(players[i] for i in range(len(features)) if positions[i] == pos) <= pos_available[pos] # Position Limit

for club in CLUBS:
  prob += lpSum(players[i] for i in range(len(features)) if teams[i] == club) <= 3 # Club Limit

In [19]:
# Solve the problem
prob.solve()

1

In [20]:
player_names = []
player_clubs = []
player_positions = []
player_prices = []
player_points = []

In [21]:
for v in prob.variables():
  if v.varValue != 0:
    name = features.name[int(v.name.split("_")[1])]
    club = features.team[int(v.name.split("_")[1])]
    position = features.position[int(v.name.split("_")[1])]
    point = features.total_points[int(v.name.split("_")[1])]
    price = features.value[int(v.name.split("_")[1])]

    player_names.append(name)
    player_clubs.append(club)
    player_positions.append(position)
    player_prices.append(price)
    player_points.append(point)
    
    print(name, position, club, point, price, sep=" | ")

Bukayo Saka | MID | ARS | 179 | 80
Gabriel dos Santos Magalhães | DEF | ARS | 146 | 50
James Maddison | MID | LEI | 181 | 80
Virgil van Dijk | DEF | LIV | 183 | 65
Alisson Ramses Becker | GK | LIV | 176 | 55
Trent Alexander-Arnold | DEF | LIV | 208 | 75
João Cancelo | DEF | MCI | 201 | 70
Bernardo Veiga de Carvalho e Silva | MID | MCI | 155 | 70
Matty Cash | DEF | AVL | 147 | 50
James Ward-Prowse | MID | SOU | 159 | 65
Michail Antonio | FWD | WHU | 140 | 75
Jarrod Bowen | MID | WHU | 206 | 85
José Malheiro de Sá | GK | WOL | 146 | 50
Ivan Toney | FWD | BRE | 139 | 70
Bryan Mbeumo | FWD | BRE | 119 | 60


In [22]:
squad = pd.DataFrame({"name": player_names, "position": player_positions, "club": player_clubs, "price": player_prices, "point": player_points})

In [23]:
squad.sort_values(by="position")

,name,position,club,price,point
1,Gabriel dos Santos Magalhães,DEF,ARS,50,146
3,Virgil van Dijk,DEF,LIV,65,183
5,Trent Alexander-Arnold,DEF,LIV,75,208
6,João Cancelo,DEF,MCI,70,201
8,Matty Cash,DEF,AVL,50,147
10,Michail Antonio,FWD,WHU,75,140
13,Ivan Toney,FWD,BRE,70,139
14,Bryan Mbeumo,FWD,BRE,60,119
4,Alisson Ramses Becker,GK,LIV,55,176
12,José Malheiro de Sá,GK,WOL,50,146


In [24]:
squad["point"].sum()

2485

In [25]:
score = str(prob.objective)
constraint = [str(const) for const in prob.constraints.values()][0]
for v in prob.variables():
  score = score.replace(v.name, str(v.varValue))
  constraint = constraint.replace(v.name, str(v.varValue))

score_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', score) )
constraint_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', constraint) )


# print("Constraint: ")
# print(constraint_pretty + " = " + str(eval(constraint_pretty)))
# print()
# print("Score: ")
# print(score_pretty + " = " + str(eval(score_pretty)))